In [1]:
## imports
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import copy as cp
import pandas as pd
import countryinfo
import geopandas as gpd
from timeit import default_timer as timer # try to measure time
from climada.engine import Impact
from os import mkdir, remove, rmdir



In [3]:
##function definition
from functions import *
from climada_functions import *
from constants import * 

In [4]:
print(cmip6vars)

{'sfcWindmax': 'SWM', 'sfcWind': 'SW', 'psl': 'SLP', 'tas': 'TAS', 'ua': 'UA', 'ta': 'TA', 'tos': 'TOS'}


In [5]:
## select data
#select variable (cmip6 naming)
selvar = 'sfcWindmax'
pathinvar = pathcmip6+'sfcWindmax/'

#preprocessing 
gst_fact = 1.67
qt = 0.98
cut=5E5
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

##climada constants
haz_type = 'WS'
haz_id = 1

## naming
#name base (meteo) variable
metvar = [cmip6vars[selvar]]
spaceres = ["br_rg"] #base resolution regridded
timeres = ["day"]
domain = ["EU"]
season = ["winE"]
scen = ["allscens"]
sep = "_"
lst_bn = metvar+spaceres+timeres+domain+season
basenamemet = sep.join(lst_bn)

#preproc field
processings = ["qt"+str(qt)[-2:]+"pst","cutarea"+format(cut,'.0E').replace("+0",''),"gst1-67"]

basenamemet_proc = make_fn(processings,basenamemet)


In [34]:
basenamemet_proc

'qt98pst_cutarea1E5_gst1-67_SWM_br_rg_day_EU_winE'

In [35]:
scenlist

['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

In [36]:
reglist2

['BI', 'IP', 'FR', 'ME', 'MD', 'SC', 'EA']

In [37]:
impflist

['CubEOT', 'Em2011', 'Wk2021']

In [23]:
#select vars
modlist = modlist_allscen+modlist_ssp585
impf_used = ['Cubic excess-over-threshold']
#impf_used = ['Emanuel']

pp_func_dic = pp_func_dic_diff
#scen_used= ['ssp126','ssp245','ssp370']
scen_used= scenlist
sel_reg = ["EU"]  #select a region
pastname = 'historical'
timeres='day'
nmems = 3

In [24]:
impf_used

['Cubic excess-over-threshold']

In [25]:
#iterate over data to remove selected files
rmhaz = True
rmimpcsv = True
rmimpmat = True
stack = True
caltype = 'AAI_EMDAT_100mn'
rp = None
if_id = 0
df_list = []
impf_keys = []

#initiate df
if stack:
    nmems=1

for impf in impf_used: 
    pp_funcname = str(pp_func_dic[impf]).split(" ")[1]
    impf_namesht = impf_sht_names[impf]

    for scen in scen_used:
        for modid, modname in enumerate(modlist):    

            #iterate over the members
            for nmem in range(nmems):
        
                for reg in sel_reg:
                
                    ##save files
                    if stack:
                        simname = 'stacked'
                        savesubdir = 'stacked/'
                    else:
                        simname = 'nmem'+str(nmem)
                        savesubdir = 'memsep/'
                    if rp:
                        simname = simname+"_rp"+str(rp)
                    
                    if reg == 'EU':
                        basepath="aggregated/"                  
                        savesubdir = savesubdir+scen+'/'
                        ##file names
                        savenamehaz = make_fn(['haz',simname,scen,modname,pp_funcname],basenamemet_proc,filetype='.h5')
                        
                        savenameimp = make_fn(['imp',simname,scen,modname,impf_namesht,caltype],basenamemet_proc)
                        
                    else: 
                        basepath="regional/"
                        savesubdir =  savesubdir+reg+'/'+scen+'/'
                        ##file names
                        savenamehaz = make_fn(['haz',simname,reg,scen,modname,pp_funcname],basenamemet_proc,filetype='.h5')
                        
                        savenameimp = make_fn(['imp',simname,reg,scen,modname,impf_namesht,caltype],basenamemet_proc)
                        
                    
                    #save hazards
                    if rmhaz:
                        try:
                            remove(pathhaz+savesubdir+savenamehaz)
                            print("removed file: "+pathhaz+savesubdir+savenamehaz)

                        except FileNotFoundError:
                            print("no file to remove")
                    #save impacts
                    if rmimpcsv:
                        try:
                            remove(pathimp+'impact csv/'+basepath+savesubdir+savenameimp+'.csv')
                            print("removed file: "+pathimp+'impact csv/'+basepath+savesubdir+savenameimp+'.csv')

                        except FileNotFoundError:
                            print("no file to remove")
                    if rmimpmat:
                        try:
                            remove(pathimp+'impact matrices/'+basepath+savesubdir+savenameimp+'.npz')
                            print("removed file: "+pathimp+'impact matrices/'+basepath+savesubdir+savenameimp+'.npz')

                        except FileNotFoundError:
                            print("no file to remove")



no file to remove
removed file: /home/lseverino/MT/scripts/results/impacts/impact csv/aggregated/stacked/historical/imp_stacked_historical_CanESM5_CubEOT_AAI_EMDAT_100mn_qt98pst_cutarea5E5_gst1-67_SWM_br_rg_day_EU_winE.csv
removed file: /home/lseverino/MT/scripts/results/impacts/impact matrices/aggregated/stacked/historical/imp_stacked_historical_CanESM5_CubEOT_AAI_EMDAT_100mn_qt98pst_cutarea5E5_gst1-67_SWM_br_rg_day_EU_winE.npz
no file to remove
removed file: /home/lseverino/MT/scripts/results/impacts/impact csv/aggregated/stacked/historical/imp_stacked_historical_CNRM-CM6-1_CubEOT_AAI_EMDAT_100mn_qt98pst_cutarea5E5_gst1-67_SWM_br_rg_day_EU_winE.csv
removed file: /home/lseverino/MT/scripts/results/impacts/impact matrices/aggregated/stacked/historical/imp_stacked_historical_CNRM-CM6-1_CubEOT_AAI_EMDAT_100mn_qt98pst_cutarea5E5_gst1-67_SWM_br_rg_day_EU_winE.npz
no file to remove
removed file: /home/lseverino/MT/scripts/results/impacts/impact csv/aggregated/stacked/historical/imp_stacked_

In [18]:
pathimp+'impact matrices/'+basepath+savesubdir+savenameimp+'.npz'

'/home/lseverino/MT/scripts/results/impacts/impact matrices/aggregated/stacked/ssp585/imp_stacked_rp20_ssp585_ACCESS-CM2_ScSig_AAI_EMDAT_100mn_qt98pst_cutarea5E5_gst1-67_SWM_br_rg_day_EU_winE.npz'